<a href="https://colab.research.google.com/github/riyagpt0251/TextOCR/blob/main/TextOCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y tesseract-ocr
!pip install pytesseract opencv-python pdf2image


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


import necessary libraries

In [11]:
import cv2
import os
import pytesseract
import numpy as np
import matplotlib.pyplot as plt
from pdf2image import convert_from_path
from IPython.display import display, Image


fuction to process ocr on image

In [3]:
def ocr_image(image_path):
    # Read Image
    image = cv2.imread(image_path)

    # Convert to Grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Thresholding (Preprocessing)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Show Preprocessed Image
    plt.imshow(thresh, cmap='gray')
    plt.title("Preprocessed Image for OCR")
    plt.show()

    # Perform OCR
    text = pytesseract.image_to_string(thresh)

    return text


upload json file

In [9]:
from google.colab import files
uploaded = files.upload()  # Select multiple images


Saving whale_test_ocr.csv to whale_test_ocr (3).csv


In [12]:
# Load CSV file
csv_file = "whale_test_ocr.csv"  # Change this if needed
df = pd.read_csv(csv_file)

# Define the folder where images are stored
image_folder = "/content/"  # Update this if images are in Drive

# Function to apply OCR
def ocr_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: {image_path} not found!")
        return None
    text = pytesseract.image_to_string(img)
    return text

# Perform OCR on each image
df["ocr_text"] = df["image"].apply(lambda img: ocr_image(os.path.join(image_folder, img)))

# Save updated CSV with OCR results
df.to_csv("whale_test_ocr_results.csv", index=False)
print("✅ OCR results saved to whale_test_ocr_results.csv")


Error: /content/bf681632d.jpg not found!
Error: /content/03f41c1f2.jpg not found!
Error: /content/f256dfa87.jpg not found!
Error: /content/3e3cbcf35.jpg not found!
Error: /content/051c05e52.jpg not found!
Error: /content/405772e8e.jpg not found!
Error: /content/cef1ce25f.jpg not found!
Error: /content/84e7fcaaa.jpg not found!
Error: /content/3d0d0dee1.jpg not found!
Error: /content/4e4110a8c.jpg not found!
Error: /content/0613d30ca.jpg not found!
Error: /content/da1228217.jpg not found!
Error: /content/857e9dcab.jpg not found!
Error: /content/866f92b2d.jpg not found!
Error: /content/1d647f03c.jpg not found!
Error: /content/77a58808f.jpg not found!
Error: /content/619393b59.jpg not found!
Error: /content/0ae8027fa.jpg not found!
Error: /content/dbc1a5e08.jpg not found!
Error: /content/7c1f843ec.jpg not found!
Error: /content/c39409b25.jpg not found!
Error: /content/2959d1993.jpg not found!
Error: /content/5d7277fa1.jpg not found!
Error: /content/69e96a823.jpg not found!
Error: /content/

In [13]:
df_results = pd.read_csv("whale_test_ocr_results.csv")
print(df_results.head())  # View first few rows


           image  line_count  \
0  bf681632d.jpg           1   
1  03f41c1f2.jpg           1   
2  f256dfa87.jpg           5   
3  3e3cbcf35.jpg           2   
4  051c05e52.jpg           3   

                                                text extract  ocr_text  
0                      #3719 College of the Atlantic   #3719       NaN  
1          #3609 Memorial University of Newfoundland   #3609       NaN  
2  519096—29/26 ”Alb 10980 in ridW W\nTODD CHANDL...     NaN       NaN  
3  TEC01—#0/06 ID 11119\nTODD CHANDLER Sﬁ 31—1\n2...     NaN       NaN  
4                    «$317 ':\n\nHWC #0336 Champagne   #0336       NaN  


data cleaning

In [14]:
import re

# Function to clean text
def clean_text(text):
    if pd.isna(text):  # Handle NaN values
        return ""
    text = re.sub(r'\n+', ' ', text)  # Remove multiple newlines
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

# Apply cleaning
df_results["ocr_text"] = df_results["ocr_text"].apply(clean_text)

# Save cleaned results
df_results.to_csv("whale_test_ocr_cleaned.csv", index=False)
print("✅ Cleaned OCR results saved to whale_test_ocr_cleaned.csv")


✅ Cleaned OCR results saved to whale_test_ocr_cleaned.csv


perform NLP

In [15]:
!pip install spacy
import spacy

# Load SpaCy NLP model
nlp = spacy.load("en_core_web_sm")

# Function to extract named entities
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Apply NER to OCR text
df_results["entities"] = df_results["ocr_text"].apply(extract_entities)

# Save results
df_results.to_csv("whale_test_ocr_entities.csv", index=False)
print("✅ Named Entities extracted and saved!")
files.download("whale_test_ocr_entities.csv")


✅ Named Entities extracted and saved!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>